In [22]:
import numpy as np
import nibabel as nb
%pylab inline
from os.path import join
import seaborn as sn
import nighres
import nilearn as nl
from nilearn import plotting
import math
import matplotlib.pyplot as plt
import os
from os.path import join
from glob import glob
import pathos.multiprocessing as multiprocessing
from functools import partial
import subprocess
import pandas as pd
from sklearn.linear_model import LinearRegression

Populating the interactive namespace from numpy and matplotlib


/opt/quarantine/anaconda3/2018.12/install/lib/python3.7/site-packages/IPython/core/magics/pylab.py:160: UserWarning: pylab import has clobbered these variables: ['plotting']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [23]:
def get_sub_data(in_dir, sub_id):
    """
    Loads an individual subject's data for all modalities
    
    - in_dir {str} : sdsdsds
    """
    
    img1 = nb.load(join(in_dir,'{}_FA_reg.nii.gz'.format(sub_id)))
    img2 = nb.load(join(in_dir,'{}_MD_reg.nii.gz'.format(sub_id)))
    img3 = nb.load(join(in_dir,'{}_MTsat.nii.gz'.format(sub_id)))
    img4 = nb.load(join(in_dir,'{}_PDsat.nii.gz'.format(sub_id)))
    img5 = nb.load(join(in_dir,'{}_R1.nii.gz'.format(sub_id)))
    img6 = nb.load(join(in_dir,'{}_R2s_OLS.nii.gz'.format(sub_id)))
    
    d1 = img1.get_data()
    d2 = img2.get_data()
    d3 = img3.get_data()
    d4 = img4.get_data()
    d5 = img5.get_data()
    d6 = img6.get_data()
        
#     d = [d1,d2,d3,d4,d5,d6]
#     m = [mk>0 for mk in d]
#     mask = np.ones_like(m[0])
#     for iii in m:
#         mask = mask*iii

    d = np.stack((d1,d2,d3,d4,d5,d6),axis=3)
    mask = np.prod(d>0,axis=3).astype(bool)
    
    return {'data':d,'mask':mask,'img':img1}

# Creating White Matter Segmentations

In [5]:
out_dir = '/data/neuralabc/carfra/QuantMetComp/source/masks_created/'
in_dir = '/data/neuralabc/source/MPI_CBS/MPM_DTI/source/'
spm_dir = '/data/neuralabc/source/MPI_CBS/MPM_DTI/processing/segmentations_MTSat_SPM/'
mgdm_dir = '/data/neuralabc/carfra/QuantMetComp/source/masks_created/'

all_dirs = glob(in_dir+'*')
sub_ids = [os.path.basename(x) for x in all_dirs]

In [17]:
def w_gMatterSeg(spm_dir,mgdm_dir,out_dir,sub_id):
    out_dir = join(out_dir,sub_id)
    spm_dir = join(spm_dir,sub_id)
    mgdm_dir = join(mgdm_dir,sub_id,'MGDM')
    
    #wm
    mgdm = nb.load(join(mgdm_dir,sub_id+'_mgdm-lbls.nii.gz'))
    spm = nb.load(join(spm_dir,sub_id+'_WM.nii.gz'))
    mgdmData = mgdm.get_data()
    spmData = spm.get_data()
    
    mask1 = (np.logical_or((mgdmData==47),(mgdmData==48))).astype(float)
    mask2 = (spmData>0.5).astype(float)
    mask = mask1[:,:,:,0]*mask2
    
    wm = nb.Nifti1Image(mask,affine=spm.affine,header=spm.header)
    wm.to_filename(join(out_dir,'WM.nii.gz'))
    
    #gm and subcortical
    spm = nb.load(join(spm_dir,sub_id+'_GM.nii.gz'))
    spmData = spm.get_data()
    
    mask1 = ((mgdmData==26)|(mgdmData==27)|(mgdmData==36)|(mgdmData==37)|(mgdmData==32)|(mgdmData==33)|
             (mgdmData==40)|(mgdmData==41)|(mgdmData==38)|(mgdmData==39)).astype(float)
    mask2 = ((mgdmData==36)|(mgdmData==37)|(mgdmData==32)|(mgdmData==33)|
             (mgdmData==40)|(mgdmData==41)|(mgdmData==38)|(mgdmData==39)).astype(float)
    mask3 = (spmData>0.95).astype(float)
    mask = mask1[:,:,:,0]*mask3
    
    gm = nb.Nifti1Image(mask,affine=spm.affine,header=spm.header)
    gm.to_filename(join(out_dir,'GM.nii.gz'))
    
    mask = mask2[:,:,:,0]*mask3
    s_gm = nb.Nifti1Image(mask,affine=spm.affine,header=spm.header)
    s_gm.to_filename(join(out_dir,'subcortex.nii.gz'))
    subprocess.call(["fslmaths",                     ##filtering with gaussian kernael to then remove random spaces and dots
                    join(out_dir,'subcortex.nii.gz'),
                    "-fmean",
                    join(out_dir,'subcortex.nii.gz')])
    s_gm_data = nb.load(join(out_dir,'subcortex.nii.gz')).get_data()
    s_gm_data[s_gm_data>0.6] = 1
    s_gm_data[s_gm_data<=0.6] = 0
    s_gm = nb.Nifti1Image(s_gm_data,affine=spm.affine,header=spm.header)
    s_gm.to_filename(join(out_dir,'subcortex.nii.gz'))
    subprocess.call(["fslmaths",                     
                    join(out_dir,'subcortex.nii.gz'),
                    "-fillh",
                    join(out_dir,'subcortex.nii.gz')])

In [18]:
w_gMatterSeg(spm_dir,mgdm_dir,out_dir,sub_ids[0])

In [19]:
import time
now = time.time()
for iiii in range(20):
    pool = multiprocessing.ProcessingPool(nodes=5)
    
    sub_ids_part = sub_ids[5*(iiii):5*(iiii+1)]
    extr = partial(w_gMatterSeg,spm_dir,mgdm_dir,out_dir)
    pool.map(extr,sub_ids_part)
    
    pool.close()
    #Needed to completely destroy the pool so that pathos doesn't reuse
    pool.clear()
    
extr = partial(w_gMatterSeg,spm_dir,mgdm_dir,out_dir)
pool.map(extr,[sub_ids[100]])

pool.close()
    #Needed to completely destroy the pool so that pathos doesn't reuse
pool.clear()
then = time.time()
print(then-now)

95.44053053855896


# Multiple regression

In [4]:
reg_dir = '/data/neuralabc/carfra/QuantMetComp/processing/MPM/MPM_correlations/GM_vs_WM/'
data_WM = pd.read_csv(join(reg_dir,'WM.csv'), index_col=0)
data_GM = pd.read_csv(join(reg_dir,'GM.csv'), index_col=0)

#reg_dir = '/data/neuralabc/carfra/QuantMetComp/processing/MPM/MPM_correlations/Cortical_vs_subcortical/'
#data_scort = pd.read_csv(join(reg_dir,'subcortical_GM.csv'), index_col=0)
#data_cort = pd.read_csv(join(reg_dir,'cortical_sheath.csv'), index_col=0)

/opt/quarantine/anaconda3/2018.12/install/lib/python3.7/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [15]:
name = ['FA','MD','MTsat','PDsat','R1','$R2^*$']

from scipy.stats import zscore
from scipy import stats

data_WM = data_WM.apply(zscore)
data_GM = data_GM.apply(zscore)

df_WM = data_WM[name[2:6]]
df_GM = data_GM[name[2:6]]

X_WM = df_WM.values.reshape(-1, 4)
X_GM = df_GM.values.reshape(-1, 4)



In [16]:
reg_WM_FA = LinearRegression()  # create object
reg_WM_MD = LinearRegression()
reg_GM_FA = LinearRegression()
reg_GM_MD = LinearRegression()

reg_WM_FA.fit(X_WM, data_WM[name[0]].values.reshape(-1, 1))  # perform linear regression
reg_WM_MD.fit(X_WM, data_WM[name[1]].values.reshape(-1, 1))
reg_GM_FA.fit(X_GM, data_GM[name[0]].values.reshape(-1, 1))
reg_GM_MD.fit(X_GM, data_GM[name[1]].values.reshape(-1, 1))

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [17]:
#coefficients
print(reg_WM_FA.coef_)
print(reg_WM_MD.coef_)
print(reg_GM_FA.coef_)
print(reg_GM_MD.coef_)

[[ 0.20863427 -0.23275813 -0.05554971  0.13430423]]
[[-0.08007268  0.03362025  0.00026666 -0.10134025]]
[[ 0.33515678 -0.05444773  0.11819014  0.02855422]]
[[-0.21508513 -0.0448493  -0.24887226  0.03943939]]


In [18]:
print(reg_WM_FA.intercept_)
print(reg_WM_MD.intercept_)
print(reg_GM_FA.intercept_)
print(reg_GM_MD.intercept_)

[1.04871232e-14]
[-8.82993493e-15]
[3.49968216e-15]
[1.72689443e-15]


In [20]:
#R squares
print(reg_WM_FA.score(X_WM, data_WM[name[0]].values.reshape(-1, 1)))
print(reg_WM_MD.score(X_WM, data_WM[name[1]].values.reshape(-1, 1)))
print(reg_GM_FA.score(X_GM, data_GM[name[0]].values.reshape(-1, 1)))
print(reg_GM_MD.score(X_GM, data_GM[name[1]].values.reshape(-1, 1)))

0.13554672194944672
0.024717645744861683
0.20521610016853675
0.16055331056562572


In [21]:
data_WM

,FA,MD,MTsat,PDsat,R1,$R2^*$
0,-1.966197,0.050518,-1.070867,1.263326,-1.260952,-0.161341
1,-1.532218,0.428728,-0.956708,1.323672,-1.722774,-0.920134
2,-2.001826,0.475463,-0.893601,0.386037,-0.891989,-0.560258
3,-1.501373,0.610945,-1.182652,0.851196,-1.261855,-0.668389
4,-1.675784,0.613493,-1.128763,1.149021,-1.512040,-0.701380
5,-1.249567,0.472800,-1.164600,1.580242,-1.614669,-0.884504
6,-1.535726,0.693784,-1.062982,0.926143,-1.209245,-0.694941
7,-1.637480,0.988711,-0.896102,0.353882,-0.893082,-0.455406
8,-1.305043,0.543817,-0.125441,0.436766,-0.556257,-0.187929
9,-1.253757,0.862383,0.681961,-0.038750,-0.038465,-0.018552


In [123]:
out_d = '/data/neuralabc/carfra/QuantMetComp/processing/MPM/WM'
in_path = '/data/neuralabc/source/MPI_CBS/MPM_DTI/source/'
mask_d = '/data/neuralabc/carfra/QuantMetComp/source/masks_created/'
#data_dir = 

sub_id = sub_ids[0]
out_dir = join(out_d,sub_id)
in_dir = join(in_path,sub_id)
if not os.path.exists(out_dir):
    os.makedirs(out_dir)
    print('Created your directory: {}'.format(out_dir))

dd = get_sub_data(in_dir,sub_id)
d = dd['data']

mask_file = nb.load(join(mask_d,sub_id,'WM.nii.gz'))
mask = mask_file.get_data().astype(bool)

for iii in np.arange(d.shape[-1]):
    data = d[...,iii][mask]
    if iii == 0:
        df = pd.DataFrame({name[iii]:data})
    else:
        df[name[iii]] = data
xVars = df[name[2:6]].apply(zscore).values.reshape(-1, 4)

fa_pred = linear_regressor.predict(xVars)

pred = np.zeros_like(mask).astype(float)
pred[mask] = fa_pred[:,0]
file = nb.Nifti1Image(pred,affine=mask_file.affine,header=mask_file.header)
file.to_filename(join(out_dir,'FA_predicted.nii.gz'))

fa_file = nb.load(join(in_dir,sub_id+'_FA_reg.nii.gz'))
fa_d = fa_file.get_data()
real = np.zeros_like(mask).astype(float)
real[mask] = fa_d[mask]
file = nb.Nifti1Image(real,affine=fa_file.affine,header=fa_file.header)
file.to_filename(join(out_dir,'FA_real.nii.gz'))


# Mapping masked data


In [33]:
m_dir = '/data/neuralabc/carfra/QuantMetComp/source/masks_created/'
in_dir = '/data/neuralabc/source/MPI_CBS/MPM_DTI/source/'
o_dir = '/data/neuralabc/carfra/QuantMetComp/processing/MPM/MPM_correlations/metrics/'

all_dirs = glob(in_dir+'*')
sub_ids = [os.path.basename(x) for x in all_dirs]

In [41]:
type_mask = "subcortex"
for sub_id in sub_ids:
    mask_dir = join(m_dir,sub_id,type_mask+'.nii.gz')
    mask = nb.load(mask_dir).get_data()
    data_dirs = glob(join(in_dir,sub_id,'*.nii.gz'))
    out_dir = join(o_dir,sub_id,type_mask)
    if os.path.exists(join(o_dir,sub_id,'subcortical_GM')):
        for ii in glob(join(o_dir,sub_id,'subcortical_GM/*')):
            os.remove(ii)     
        os.rmdir(join(o_dir,sub_id,'subcortical_GM'))
        
    if not os.path.exists(out_dir):
        os.makedirs(out_dir)
    for data in data_dirs:
        data_f = nb.load(data)
        data_d = data_f.get_data()
        masked_d = data_d*mask
        file = nb.Nifti1Image(masked_d,affine=data_f.affine,header=data_f.header)
        file.to_filename(join(out_dir,os.path.basename(data)))
        